In [1]:
from pyspark.sql import SQLContext


from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
import os

#os.environ['PYSPARK_SUBMIT_ARGS'] = "--master local[2] pyspark-shell"
#os.environ['JAVA_HOME'] = "/usr/lib/jvm/java-8-oracle"

sc = SparkContext()
#spark = SparkSession(sc)

sqlContext = SQLContext(sc)

#df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('appid_datapoint_full.csv')

## Reading appid_datapoint and writting as parquet

In [37]:
t1 = datetime.datetime.now()
df = sqlContext.read.format('com.databricks.spark.csv').options(header=False, inferschema=True).load('appid_datapoint_full.csv', schema=oldSchema)
t2 = datetime.datetime.now()
str(t2-t1)

'0:00:00.016930'

In [36]:
oldSchema

StructType(List(StructField(time_stamp,LongType,true),StructField(source,StringType,true),StructField(zone,StringType,true),StructField(target_address,StringType,true),StructField(application,StringType,true),StructField(applicationgroup,StringType,true),StructField(target_attribute_id,IntegerType,true),StructField(byte_count,LongType,true),StructField(tx_byte_count,LongType,true),StructField(rx_byte_count,LongType,true),StructField(flow_count,IntegerType,true),StructField(tx_flow_count,IntegerType,true),StructField(rx_flow_count,IntegerType,true),StructField(tcp_rsp_time,IntegerType,true),StructField(app_rsp_time,IntegerType,true),StructField(user_data,StringType,true)))

In [33]:
oldSchema = df.schema
t1 = datetime.datetime.now()

cols = [ 'time_stamp' , 'source' , 'zone' , 'target_address' , 'application' , 'applicationgroup' , 'target_attribute_id' , 'byte_count' , 'tx_byte_count' , 'rx_byte_count' , 'flow_count' , 'tx_flow_count' , 'rx_flow_count' , 'tcp_rsp_time' , 'app_rsp_time' , 'user_data' ]
for i,k in enumerate(oldSchema.fields):
    k.name = cols[i]
df = sqlContext.read.format("com.databricks.spark.csv").options(header='false', delimiter='\t').load("appid_datapoint_full.csv", schema=oldSchema)
t2 = datetime.datetime.now()
str(t2-t1)

#df2.write.parquet('appid_attribute_parquet')

'0:00:00.014749'

In [38]:
df.head()

Row(time_stamp=1510358400000, source='10.6.1.101', zone='\\N', target_address='134.141.77.129/', application='Salesforce', applicationgroup='Business Applications', target_attribute_id=214749637, byte_count=0, tx_byte_count=0, rx_byte_count=0, flow_count=0, tx_flow_count=0, rx_flow_count=0, tcp_rsp_time=0, app_rsp_time=0, user_data='\\N')

In [49]:
df.write.parquet('appid_datapoint_parquet1')

## Reading appid_attribute and storing as parquet

In [2]:
import datetime
#spark = SparkSession(sc)

#sqlContext = SQLContext(sc)
t2 = datetime.datetime.now()

In [16]:
df2 = sqlContext.read.format('com.databricks.spark.csv').options(header=True, inferschema=True).load('appid_attribute_full_final.csv')
t3 = datetime.datetime.now()

In [17]:
df2.head()

Row(attribute_id=214744672, time_stamp=1510358400000, source='134.141.5.104', target_address='134.141.168.108/', target_type='SERVER', location='Salem NH', detailed_location='134.141.102.17/+maclsql1a/134.141.168.108/eth1 (ge.1.9)', device_type='\\N', profile='Server Farm Default', username=None, user_data1='\\N', user_data2='\\N')

In [18]:
str(t3-t2)

'4:07:29.372480'

In [12]:
#oldSchema = df2.schema
#cols = ['target_attribute_id' , 'time_stamp' , 'source' , 'target_address' , 'target_type' , 'location' , 'detailed_location' , 'device_type' , 'profile' , 'username' , 'user_data1' , 'user_data2' ]
#for i,k in enumerate(oldSchema.fields):
#    k.name = cols[i]
#df2 = sqlContext.read.format("com.databricks.spark.csv").options(header='false', delimiter='\t').load("appid_attribute_full_final.csv", schema=oldSchema)


#df2.write.parquet('appid_attribute_parquet')

In [30]:
df2.write.parquet('appid_attribute_parquet')

### Efficiency test

####  time = 4.59

In [26]:
newDF = df.join(df2, df.target_attribute_id == df2.attribute_id, 'left')
newDF.show()

AttributeError: 'DataFrame' object has no attribute 'index'

In [27]:
newDF.head()

Row(time_stamp=None, source=None, zone=None, target_address=None, application=None, applicationgroup=None, target_attribute_id=None, byte_count=None, tx_byte_count=None, rx_byte_count=None, flow_count=None, tx_flow_count=None, rx_flow_count=None, tcp_rsp_time=None, app_rsp_time=None, user_data=None, attribute_id=None, time_stamp=None, source=None, target_address=None, target_type=None, location=None, detailed_location=None, device_type=None, profile=None, username=None, user_data1=None, user_data2=None)

In [22]:
datetime.datetime.now()

datetime.datetime(2018, 6, 14, 7, 41, 18, 535029)

In [4]:
import pyarrow.parquet as pq
import datetime
t1 = datetime.datetime.now()
df_p = pq.read_table('appid_attribute_parquet')
#df_ptod = df_p.to_pandas()
t2 = datetime.datetime.now()
print((str(t2-t1)))
#print(len(df_ptod))
       

0:00:08.087890


In [6]:
import pandas as pd
df_ptod = None
t1 = datetime.datetime.now()
df_p = pd.read_csv('appid_attribute_full_final.csv')

t2 = datetime.datetime.now()
print((str(t2-t1)))
print(len(df_p))

0:00:51.659943
21494586


In [ ]:
import pyarrow.parquet as pq
import datetime
t1 = datetime.datetime.now()
df_p = pq.read_table('appid_datapoint_parquet')
#df_ptod = df_p.to_pandas()
t2 = datetime.datetime.now()
print((str(t2-t1)))
print(len(df_ptod))
       

In [7]:
df_p = None

In [ ]:
df_p.head()

In [ ]:
df_p[(df_p.source =='10.6.1.101') & (df_p.application =='Salesforce')]

In [4]:
df_ptod.head()

,attribute_id,time_stamp,source,target_address,target_type,location,detailed_location,device_type,profile,username,user_data1,user_data2
0,214744672,1510358400000,134.141.5.104,134.141.168.108/,SERVER,Salem NH,134.141.102.17/+maclsql1a/134.141.168.108/eth1...,\N,Server Farm Default,None,\N,\N
1,214744673,1510358400000,134.141.5.104,134.141.72.72/,CLIENT,Salem NH,134.141.104.103/maeq6510-1/ctl0-eth1/iscsi2 (t...,\N,Server iSCSI Node,None,\N,\N
2,214744674,1510358400000,134.141.5.104,134.141.79.192/,SERVER,Salem NH,\N,\N,\N,\N,\N,\N
3,214744675,1510358400000,134.141.5.104,10.51.1.154/,SERVER,Other Corporate Labs,\N,\N,\N,\N,\N,\N
4,214744676,1510358400000,134.141.5.104,134.141.90.41/eospina-pc,CLIENT,Salem NH,134.141.104.209/2-78 Gray (ge.4.6),Windows,Allow Exteneded Access Salem,CORP\\daveh,\N,\N


In [27]:
#import pandas as pd
#pd_df = pd.read_csv('appid_attribute_full_final.csv')
#pd_df.head()
pd_df = None

In [19]:
#row = pd_df.columns
#pd_df.columns = ['attribute_id' , 'time_stamp' , 'source' , 'target_address' , 'target_type' , 'location' , 'detailed_location' , 'device_type' , 'profile' , 'username' , 'user_data1' , 'user_data2' ]



In [24]:
#temp = pd.DataFrame(pd.DataFrame(row).transpose())
#temp.columns = ['attribute_id' , 'time_stamp' , 'source' , 'target_address' , 'target_type' , 'location' , 'detailed_location' , 'device_type' , 'profile' , 'username' , 'user_data1' , 'user_data2' ]
#pd_df = pd_df.append(temp)


In [25]:
#pd_df.to_csv('appid_attribute_full_final.csv',index =False)

In [20]:
pd_df.head()

,attribute_id,time_stamp,source,target_address,target_type,location,detailed_location,device_type,profile,username,user_data1,user_data2
0,214744672,1510358400000,134.141.5.104,134.141.168.108/,SERVER,Salem NH,134.141.102.17/+maclsql1a/134.141.168.108/eth1...,\N,Server Farm Default,NaN,\N,\N
1,214744673,1510358400000,134.141.5.104,134.141.72.72/,CLIENT,Salem NH,134.141.104.103/maeq6510-1/ctl0-eth1/iscsi2 (t...,\N,Server iSCSI Node,NaN,\N,\N
2,214744674,1510358400000,134.141.5.104,134.141.79.192/,SERVER,Salem NH,\N,\N,\N,\N,\N,\N
3,214744675,1510358400000,134.141.5.104,10.51.1.154/,SERVER,Other Corporate Labs,\N,\N,\N,\N,\N,\N
4,214744676,1510358400000,134.141.5.104,134.141.90.41/eospina-pc,CLIENT,Salem NH,134.141.104.209/2-78 Gray (ge.4.6),Windows,Allow Exteneded Access Salem,CORP\\daveh,\N,\N


In [12]:

df =spark.read.csv(
    "appid_attribute_full_final.csv", header=True, mode="DROPMALFORMED"
)

### Reading parquet with mension size

In [2]:
from pyspark.sql import SparkSession
import datetime
# initialise sparkContext
spark = SparkSession.builder \
    .master('local') \
    .appName('p7_sample') \
    .config('spark.executor.memory', '24gb') \
    .config("spark.cores.max", "16") \
    .getOrCreate()

sc = spark.sparkContext

# using SQLContext to read parquet file
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)


In [3]:
# to read parquet file
df = sqlContext.read.parquet('appid_datapoint_parquet1')

In [13]:
t1 = datetime.datetime.now()

df1 = df[(df.source =='10.6.1.101') & (df.application =='DNS') ]
t2 = datetime.datetime.now()
print((str(t2-t1)))

0:00:00.069382


In [14]:
df1.head(5)

[Row(time_stamp=1511780400000, source='10.6.1.101', zone='\\N', target_address='10.6.16.123/', application='DNS', applicationgroup='Protocols', target_attribute_id=218165898, byte_count=0, tx_byte_count=0, rx_byte_count=0, flow_count=0, tx_flow_count=0, rx_flow_count=0, tcp_rsp_time=0, app_rsp_time=0, user_data='\\N'),
 Row(time_stamp=1511780400000, source='10.6.1.101', zone='\\N', target_address='10.6.16.32/', application='DNS', applicationgroup='Protocols', target_attribute_id=218165899, byte_count=73951948, tx_byte_count=46738951, rx_byte_count=27212997, flow_count=492880, tx_flow_count=246326, rx_flow_count=246554, tcp_rsp_time=224, app_rsp_time=244753, user_data='\\N'),
 Row(time_stamp=1511780400000, source='10.6.1.101', zone='\\N', target_address='10.6.25.30/', application='DNS', applicationgroup='Protocols', target_attribute_id=218165900, byte_count=42292759, tx_byte_count=17802393, rx_byte_count=24490366, flow_count=235158, tx_flow_count=118308, rx_flow_count=116850, tcp_rsp_ti

In [15]:
ts = df1.select('time_stamp').collect()

In [16]:
ts[0]

Row(time_stamp=1511780400000)

In [17]:
ts[0].time_stamp

1511780400000

In [22]:
t1 = datetime.datetime.now()

ts = df1.select('time_stamp').collect()
ts_array = [int(i.time_stamp) for i in ts]
y = df1.select('app_rsp_time').collect()
y_array = [int(i.app_rsp_time) for i in y]

t2 = datetime.datetime.now()
print((str(t2-t1)))

0:01:03.984618


In [23]:
import pandas as pd
df2 = pd.DataFrame({'ds':ts_array,'y':y_array})

In [27]:
#dates = df2.ds.astype(int)
dates_outlook =pd.to_datetime(pd.Series(ts_array),unit='ms')

## Parallel processing

In [26]:
import multiprocessing as mp
from joblib import Parallel, delayed

In [ ]:
pool = Parallel(n_jobs=-1,verbose=5,pre_dispatch='all')
results = pool(delayed(get_sentance_score)(doc,data_all,IG_all,bench_median_len,bench_median_len_G) for doc in doc_list) #doc_list[0:50000]
